In [6]:
!pip install -q bitsandbytes

!apt update
!apt install -y git

Get:1 http://archive.ubuntu.com/ubuntu jammy InRelease [270 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]        
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 Packages [17.5 MB]
Get:6 http://archive.ubuntu.com/ubuntu jammy/multiverse amd64 Packages [266 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy/main amd64 Packages [1792 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy/restricted amd64 Packages [164 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1599 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1602 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [49.8 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1305 kB]
Get:13 http://archive.ubuntu.com/ubuntu jam

In [7]:
!git clone https://github.com/AI4Bharat/IndicTrans2

Cloning into 'IndicTrans2'...
remote: Enumerating objects: 557, done.
remote: Counting objects: 100% (292/292), done.
remote: Compressing objects: 100% (186/186), done.
remote: Total 557 (delta 172), reused 195 (delta 105), pack-reused 265
Receiving objects: 100% (557/557), 4.08 MiB | 20.36 MiB/s, done.
Resolving deltas: 100% (326/326), done.


In [8]:
%cd IndicTrans2/huggingface_inference

/lab/IndicTrans2/huggingface_inference


/usr/local/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [9]:
%ls

IndicTransTokenizer/                         example.py
README.md                                    install.sh
configuration_indictrans.py                  modeling_indictrans.py
convert_indictrans_checkpoint_to_pytorch.py


In [10]:
# Install all the dependencies and requirements associated with the project for running HF compatible models.
!source install.sh

Setting up the environment in the /lab/IndicTrans2/huggingface_inference
Creating a virtual environment with python3
install.sh: line 10: conda: command not found
install.sh: line 11: conda: command not found
Installing all the dependencies
install.sh: line 14: conda: command not found
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (139/139), 149.77 MiB | 36.24 MiB/s, done.
Resolving deltas: 100% (53/53), done.
Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 1399, done.
remote: Counting objects: 100% (180/180), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 1399 (delta 135), reused 147 (delta 120), pack-reused 1219
Receiving objects: 100% (1399/1399), 9.

In [15]:
%%time 

import sys
import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig
from IndicTransTokenizer.utils import preprocess_batch, postprocess_batch
from IndicTransTokenizer.tokenizer import IndicTransTokenizer

en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-1B" # ai4bharat/indictrans2-en-indic-dist-200M
indic_en_ckpt_dir = "ai4bharat/indictrans2-indic-en-1B" # ai4bharat/indictrans2-indic-en-dist-200M
indic_indic_ckpt_dir = "ai4bharat/indictrans2-indic-indic-1B"   # ai4bharat/indictrans2-indic-indic-dist-320M
BATCH_SIZE = 4
# BATCH_SIZE = 64
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

if len(sys.argv) > 1:
    quantization = sys.argv[1]
else:
    quantization = ""


def initialize_model_and_tokenizer(ckpt_dir, direction, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = IndicTransTokenizer(direction=direction)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig
    )
    
    if qconfig==None:
        model = model.to(DEVICE)
        model.half()
    
    model.eval()
    
    return tokenizer, model


def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer):
    translations = []
    for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]

        # Preprocess the batch and extract entity mappings
        batch, entity_map = preprocess_batch(
            batch, src_lang=src_lang, tgt_lang=tgt_lang
        )

        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            src=True,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        # Generate translations using the model
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode the generated tokens into text
        generated_tokens = tokenizer.batch_decode(
            generated_tokens.detach().cpu().tolist(), src=False
        )

        # Postprocess the translations, including entity replacement
        translations += postprocess_batch(
            generated_tokens, lang=tgt_lang, placeholder_entity_map=entity_map
        )

        del inputs
        torch.cuda.empty_cache()

    return translations


en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(
    en_indic_ckpt_dir, "en-indic", quantization
)
# indic_en_tokenizer, indic_en_model = initialize_model_and_tokenizer(
#     indic_en_ckpt_dir, "indic-en", quantization
# )
# indic_indic_tokenizer, indic_indic_model = initialize_model_and_tokenizer(
#     indic_indic_ckpt_dir, "indic-indic", quantization
# )


# ---------------------------------------------------------------------------
#                              English to Hindi
# ---------------------------------------------------------------------------
en_sents = [
    "When I was young, I used to go to the park every day.",
    "He has many old books, which he inherited from his ancestors.",
    "I can't figure out how to solve my problem.",
    "She is very hardworking and intelligent, which is why she got all the good marks.",
    "We watched a new movie last week, which was very inspiring.",
    "If you had met me at that time, we would have gone out to eat.",
    "She went to the market with her sister to buy a new sari.",
    "Raj told me that he is going to his grandmother's house next month.",
    "All the kids were having fun at the party and were eating lots of sweets.",
    "My friend has invited me to his birthday party, and I will give him a gift.",
]
src_lang, tgt_lang = "eng_Latn", "hin_Deva"
hi_translations = batch_translate(
    en_sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer
)

print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(en_sents, hi_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")


/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config.json: 100%|██████████| 1.27k/1.27k [00:00<00:00, 2.15MB/s]
configuration_indictrans.py: 100%|██████████| 14.1k/14.1k [00:00<00:00, 14.5MB/s]
A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-1B:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
modeling_indictrans.py: 100%|██████████| 58.6k/58.6k [00:00<00:00, 23.2MB/s]
A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-1B:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading 


eng_Latn - hin_Deva
eng_Latn: When I was young, I used to go to the park every day.
hin_Deva: जब मैं छोटा था, मैं हर दिन पार्क जाता था।
eng_Latn: He has many old books, which he inherited from his ancestors.
hin_Deva: उनके पास कई पुरानी किताबें हैं, जो उन्हें अपने पूर्वजों से विरासत में मिली हैं।
eng_Latn: I can't figure out how to solve my problem.
hin_Deva: मुझे समझ नहीं आ रहा है कि मैं अपनी समस्या का समाधान कैसे करूं।
eng_Latn: She is very hardworking and intelligent, which is why she got all the good marks.
hin_Deva: वह बहुत मेहनती और बुद्धिमान है, यही कारण है कि उसे सभी अच्छे अंक मिले।
eng_Latn: We watched a new movie last week, which was very inspiring.
hin_Deva: हमने पिछले हफ्ते एक नई फिल्म देखी, जो बहुत प्रेरणादायक थी।
eng_Latn: If you had met me at that time, we would have gone out to eat.
hin_Deva: अगर आप उस समय मुझसे मिलते तो हम बाहर खाना खाने जाते।
eng_Latn: She went to the market with her sister to buy a new sari.
hin_Deva: वह अपनी बहन के साथ नई साड़ी खरीदने के लिए बाजार 

In [16]:
import os
import csv
import time
from datasets import load_dataset

In [17]:
possible_configs = [
#     # "main",
#     # "socratic",
    'ARC-Challenge',
    'ARC-Easy'
]
dataset = []
for config in possible_configs:
    dataset_slice = load_dataset("ai2_arc", config,ignore_verifications=True)
    dataset.append(dataset_slice)
    

/usr/local/lib/python3.11/site-packages/datasets/load.py:2487: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(
Generating validation split: 100%|██████████| 570/570 [00:00<00:00, 142936.34 examples/s]


In [18]:
%%time 
en_sents = ['how are you kartik','i am good']
hi_translations = batch_translate(
    en_sents, "eng_Latn", "hin_Deva", en_indic_model, en_indic_tokenizer
)
print(hi_translations)

['कार्तिक, आप कैसे हैं?', 'मैं अच्छा हूँ।']
CPU times: user 170 ms, sys: 10 ms, total: 180 ms
Wall time: 201 ms


In [19]:
%%time
from functools import partial
for i in range(len(possible_configs)):
    for set in dataset[i]:
        set_list = []
        columns = ['question','choices']
        
        for col in columns:
            values = [str(item[col]) for item in dataset[i][set]]
            # print(values[1:2])
            
            # Use ThreadPoolExecutor for parallel translation
            
            if __name__ == '__main__':
                result =[]
                result = batch_translate(values[:20], "eng_Latn", "hin_Deva", en_indic_model, en_indic_tokenizer)
                    
            set_list.append(result)

        # Create folders for each configuration
        current_directory = os.getcwd()
        
        # Specify the path of the 'config' folder
        config_folder_path = os.path.join(current_directory, possible_configs[i])
        
        # Create the 'config' folder
        os.makedirs(config_folder_path, exist_ok=True)       
    
        # Transpose the 2D list
        transposed_data = list(map(list, zip(*set_list)))
        path = os.path.join(possible_configs[i], f'{set}.csv')
        with open(path, 'w', encoding='utf-8') as f:
            # using csv.writer method from CSV package
            write = csv.writer(f)
            # write.writerow(columns)
            write.writerows(transposed_data)

CPU times: user 1min 25s, sys: 1.87 s, total: 1min 27s
Wall time: 1min 32s
